<a href="https://colab.research.google.com/github/lucasestrela/Dissertacao/blob/main/Rodando_Modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1) Importando pacotes

In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# 2) Importando os dados

Irei utilizar os custos do ciclo calculados e as informações dos países.

In [5]:
#############################################################################
######################## --- Lendo Base de Dados --- ########################
#############################################################################
custo = pd.read_csv('https://raw.githubusercontent.com/lucasestrela/Dissertacao/main/Dados/custos_data.csv',
                    header = 0,
                    index_col = 0
                    )


dados_paises = pd.read_csv('https://raw.githubusercontent.com/lucasestrela/Dissertacao/main/Dados/countries_data.csv',
                    header = 0,
                    index_col = 0
                    )

Faço o merge dos dataframes

In [6]:
# Fazendo o merge dos dois datasets
dataframe = pd.merge(custo, dados_paises, on='countrycode', left_index=True, right_index=True, how='left')
dataframe[['log_gdp_percapta', 'log_emp_pop', 'log_labsh']] = np.log(dataframe[['gdp_percapta', 'emp_pop', 'labsh']])

# Apagando as outras variáveis
del custo, dados_paises

In [8]:
dataframe['custo']

0        3.629999
1       69.708880
2       48.805136
3             NaN
4       76.796450
          ...    
178      7.100896
179     28.649199
180      1.150329
181     24.514667
182    573.362084
Name: custo, Length: 183, dtype: float64

# 3) Rodando o Modelo

Para o python, é válido separar as variáveis entre X (covariadas) e y (variável independente) em dois dataframes separados.

In [7]:
##############################################################################
######################## --- Separando covariadas --- ########################
##############################################################################

# X = dataframe[['lvaw_garriga', 'stab_n', 'stab_s', 'gdp_percapta', 'emp_pop', 'avh', 'hc', 'rwtfpna', 'labsh']]
X = dataframe[['lvaw_garriga', 'stab_n', 'stab_s', 'log_gdp_percapta', 'log_emp_pop', 'log_labsh']]
X = sm.add_constant(X) ## let's add an intercept (beta_0) to our model

y = dataframe['custo']


# Rodando o modelo
model = sm.OLS(y, X, missing='drop').fit(cov_type='HC0')

# Printando os resultados
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  custo   R-squared:                       0.189
Model:                            OLS   Adj. R-squared:                  0.150
Method:                 Least Squares   F-statistic:                    0.2516
Date:                Wed, 10 Mar 2021   Prob (F-statistic):              0.958
Time:                        02:42:12   Log-Likelihood:                -2309.1
No. Observations:                 131   AIC:                             4632.
Df Residuals:                     124   BIC:                             4652.
Df Model:                           6                                         
Covariance Type:                  HC0                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------
const            -1.798e+07   1.48e+07  